### Computing the Gibbs free energy with equilibrator-api

In [1]:
from equilibrator_api import ComponentContribution, Reaction, ReactionMatcher
import numpy as np
import pandas as pd
import cobra
from cobra.flux_analysis.variability import flux_variability_analysis
import cvxopt
import json
# Load equilibrator data
eq_api = ComponentContribution(pH=7.5, ionic_strength=0.25)  # loads data

In [2]:
# Get iJO1366 reactions as KEGG reaction strings
iJO1366ToKEGG = pd.read_csv(r'C:\Users\tinta\OneDrive\Documents\Projects\Ordering_of_Concentrations\equilibrator-api\src\equilibrator_api\data\iJO1366_reactions.csv',
                           index_col=0)
iJO1366ToKEGG.head()

,formula
bigg.reaction,
ex_14glucan_e,C00718 <=>
ex_fe3hox_e,C06227 <=>
ex_mobd_e,C06232 <=>
14glucanabcpp,C00001 + C00002 + C00718 <=> C00008 + C00009
14glucantexi,<=> C00718


In [3]:
# You control the pH and ionic strength!
# ionic strength is in Molar units.
rxn_name = '23pde2pp'
rxn = Reaction.parse_formula(iJO1366ToKEGG.loc[rxn_name].item())
dG0_prime, dG0_uncertainty = eq_api.dG0_prime(rxn)
print("dG0' = %.1f \u00B1 %.1f kJ/mol\n" % (dG0_prime, dG0_uncertainty))

dG0' = -38.5 ± 2.8 kJ/mol



In [3]:
iJO1366 = cobra.io.load_json_model('iJO1366.json')
# Make internal reactions reversible

# remove blocked reactions
blockedRxns = cobra.flux_analysis.find_blocked_reactions(
    iJO1366, zero_cutoff=1e-9, open_exchanges=False)

for rxn in blockedRxns:
    iJO1366.reactions.get_by_id(rxn).remove_from_model(remove_orphans=True)
    
iJO1366_rxns = [rxn.id.lower() for rxn in iJO1366.reactions]

In [ ]:

    def __enableUptakeOfCarbonSources(self):
        if self.name == 'iJO1366':
            self.GEM.reactions.get_by_id('EX_glyc__R_e').lower_bound = -1000
            self.GEM.reactions.get_by_id('EX_ac_e').lower_bound = -1000
            
            
                def updateExchangeReactionBounds(self, carbonSource=None, carbonUptakeRate=20):
        """
        Update exchange reaction bounds to simulate appropriate growth medium
        conditions.
        """
        if self.name == 'iJO1366':
            ExchangeRxnIDs = [rxn.id for rxn in self.GEM.exchanges if len(rxn.reactants) == 0]
            for ID in ExchangeRxnIDs:
                try:
                    if self.__isOrganicExchange(ID):
                        self.GEM.reactions.get_by_id(ID).upper_bound = 0
                except Exception:
                    pass
            self.__setEcoliCarbonSourceUptake(carbonSource, carbonUptakeRate)
            self.carbonSource = carbonSource
        else:
            if carbonSource is not None:
                warnings.warn('Carbon source selection only available for model iJO1366')
            if self.name == 'EvAraModel':
                sucroseUptake = 'EX_cpd00076_e0_reverse'
                self.GEM.reactions.get_by_id(sucroseUptake).upper_bound = carbonUptakeRate
            else:
                warnings.warn('Only implemented for iJO1366 and EvAraModel')

    def __isOrganicExchange(self, ID):
        compoundAtoms = list(self.GEM.reactions.get_by_id(ID).products[0].formula)
        cond = (('C' in compoundAtoms)
                & ('H' in compoundAtoms)
                & ('o' not in compoundAtoms))  # discards cobalamine
        return cond

    def __setEcoliCarbonSourceUptake(self, carbonSource, carbonUptakeRate):
        """
        Set uptake rate for the selected carbon source for the E. coli model (iJO1366)
        """
        carbonSource = carbonSource.lower()
        if carbonSource in 'glucose':
            uptakeRxns = ['EX_glc__D_e_reverse']
        elif carbonSource in 'acetate':
            uptakeRxns = ['EX_ac_e_reverse']
        elif carbonSource in 'glycerate':
            uptakeRxns = ['EX_glyc__R_e_reverse']
        elif carbonSource in 'all':
            uptakeRxns = ['EX_glc__D_e_reverse', 'EX_ac_e_reverse', 'EX_glyc__R_e_reverse']
        for rxn in uptakeRxns:
            self.GEM.reactions.get_by_id(rxn).upper_bound = carbonUptakeRate
            
            
            
        self.GEM.objective = self.GEM.reactions.get_by_any(ReactionID)
        vbioMax = self.GEM.optimize().objective_value
        print('Maximum growth rate: ' + str(vbioMax) + ' h^{-1}')


In [4]:
# Prepare dictionary with dG0
rxnGibbs = {}
for rxn_id in iJO1366_rxns:
    try:
        rxn = Reaction.parse_formula(iJO1366ToKEGG.loc[rxn_id].item())
        dG0_prime, dG0_uncertainty = eq_api.dG0_prime(rxn)
        rxnGibbs[rxn_id] = [dG0_prime, dG0_uncertainty]
    except Exception:
        pass

In [6]:
len(rxnGibbs)

427

In [5]:
# Constants
# dG0 in kJ/mol
R = 8.3144598 * 1e-3 # kJ. K^-1. mol^-1
T = 310.15 # K

## Find irreversible reactions under growth on glucose

In [6]:
# Might be necessary to use the loopless option to avoid the inclusion of
# thermodynamically infeasible loops which could render the LP infeasible
# fva = flux_variability_analysis(iJO1366, fraction_of_optimum=0.75,
#                                loopless=True, processes=2).round(decimals=4)
# fva.head()

fva = pd.read_csv('iJO1366fva.csv', index_col=0)
fva.head()

,minimum,maximum
DM_4crsol_c,0.0002,0.0002
DM_5drib_c,0.0002,0.0002
DM_amob_c,0.0000,0.0000
DM_mththf_c,0.0004,0.0004
DM_oxam_c,0.0000,-0.0000


In [7]:
# loop over fva limits
Irr_rxns = {}
for rxn_id in fva.index:
    v_min, v_max = fva.loc[rxn_id].minimum, fva.loc[rxn_id].maximum
    if v_min < 0 and v_max <= 0:
        Irr_rxns[rxn_id.lower()] = 'backward'
    elif v_min >= 0 and v_max > 0:
        Irr_rxns[rxn_id.lower()] = 'forward'

In [8]:
# Get stoichiometric matrix (irreversible reactions with Gibb's energy data)
GEM = iJO1366.copy()

def notIrreversibleWithGibbsEnergy(rxn):
    uncertainty_threshold = alpha = 0.25
    rxn_id = rxn.id.lower()
    cond = ( (rxn_id not in Irr_rxns.keys())
            or (rxn_id in Irr_rxns.keys() and rxn_id not in rxnGibbs.keys())
            or (rxn_id in rxnGibbs.keys() 
                and rxnGibbs[rxn_id][1] > alpha * rxnGibbs[rxn_id][0]) )
    return cond

for rxn in iJO1366.reactions:
    if notIrreversibleWithGibbsEnergy(rxn):
        GEM.reactions.get_by_id(rxn.id).remove_from_model(remove_orphans=False)

N = cobra.util.array.create_stoichiometric_matrix(GEM, array_type='dense')
Irr_rxns_with_dG0 = [rxn.id.lower() for rxn in GEM.reactions]
N_Irr = pd.DataFrame(data=N, index=[met.id for met in GEM.metabolites],
                     columns=Irr_rxns_with_dG0)

# Remove orphan metabolites (no reaction)
for met in N_Irr.index:
    if sum(abs(N_Irr.loc[met])) == 0:
        N_Irr = N_Irr.drop(met, axis=0)

# Change direction of backward reactions
for rxn in N_Irr.columns:
    if Irr_rxns[rxn] == 'backward':
        N_Irr[rxn] *= -1
        N_Irr.rename(columns={rxn: rxn + '_back'}, inplace=True)

N_met_pairs = int(0.5 * len(N_Irr.index) * (len(N_Irr.index) - 1))
print('There are ' + str(N_met_pairs) + ' metabolite pairs')

There are 2485 metabolite pairs


## Prepare linear program

In [14]:
# Build LP (fixed components) 

# Standard in cvxopt is Ax <= b so have to change signs and add epsilon to rhs
n_rxns, n_mets = N_Irr.values.transpose().shape

# Bounds
x_min, x_max = 1e-8, 2e-1 # molar
logx_min = (np.log(x_min)) * np.ones((n_mets, 1))
logx_max = (np.log(x_max)) * np.ones((n_mets, 1))

# Construct vectors: dG0min, dG0max
dG0min = dG0max = np.zeros((n_rxns, 1))
for i, rxn in enumerate(Irr_rxns_with_dG0):
    dG0_i, dG0_error_i = rxnGibbs[rxn]
    dG0min[i] = dG0_i - dG0_error_i
    dG0max[i] = dG0_i + dG0_error_i

Gibbs_eps = 1e-6
A0 = np.hstack((N_Irr.values.transpose(), (1 / (R*T)) * np.identity(n_rxns)))
A1 = np.hstack((np.zeros((n_rxns, n_mets)), -np.identity(n_rxns)))
A2 = np.hstack((np.zeros((n_rxns, n_mets)), np.identity(n_rxns)))
A3 = np.hstack((-np.identity(n_mets), np.zeros((n_mets, n_rxns))))
A4 = np.hstack((np.identity(n_mets), np.zeros((n_mets, n_rxns))))

A = cvxopt.matrix(np.vstack((A0, A1, A2, A3, A4)))
b = cvxopt.matrix(np.vstack((-Gibbs_eps * np.ones((n_rxns, 1)),
                             -dG0min, dG0max,
                             -logx_min, logx_max)))

# Eliminate messages
# cvxopt.solvers.options['glpk'] = {'msg_lev': 'GLP_MSG_OFF'} 

# Start iteration over metabolite pairs (variable components)
met_orders = []
for p in range(n_mets):
    for q in range(n_mets):
        if p != q:
            c = np.zeros(n_mets + n_rxns)
            c[[p, q]] = [1, -1]
            c = cvxopt.matrix(c)

            res = cvxopt.solvers.lp(c, A, b, solver='glpk',
                                    options={'glpk':{'msg_lev':'GLP_MSG_OFF'}})
            z = res['primal objective']
            if z > 0:
                met_i, met_j = N_Irr.index[[p, q]]
                min_ratio = np.e**z
                met_orders.append([met_i, met_j, min_ratio])

In [11]:
met_orders[:5]

[['3odcoa_c', '13dpg_c', 2.562942217671523],
 ['3odcoa_c', 'ac_c', 2.5629422176715186],
 ['3odcoa_c', 'accoa_c', 1.3303536033469967],
 ['3odcoa_c', 'actp_c', 56.57509098306293],
 ['3odcoa_c', 'fadh2_c', 56.57509098306293]]

## Create the transitive reduction of the graph with Networkx

In [13]:
import networkx as nx

# Build full graph adjacency matrix
total_mets = np.unique(
    np.array([[met_pairs[0], met_pairs[1]] for met_pairs in met_orders]).flatten()).tolist()
N_nodes = len(total_mets)

A = np.zeros((N_nodes, N_nodes))
for met_pair in met_orders:
    i, j = total_mets.index(met_pair[0]), total_mets.index(met_pair[1])
    A[i, j] = met_pair[2]

# Get transitive reduction of the graph
G = nx.from_numpy_matrix(A, create_using=nx.DiGraph())
G_plus = nx.transitive_reduction(G)

# Get adjacency matrix of the transitive reduction
# A_plus = nx.adjacency_matrix(G)
A_plus = nx.to_numpy_matrix(G)

# Get list of edges
reduced_met_orders = []
for i in range(N_nodes):
    for j in range(N_nodes):
        met_ratio = A_plus[i, j]
        if met_ratio > 1:
            reduced_met_orders.append([total_mets[i], total_mets[j], met_ratio])

reduced_met_orders[:5]

[['3odcoa_c', '13dpg_c', 2.562942217671523],
 ['3odcoa_c', 'ac_c', 2.5629422176715186],
 ['3odcoa_c', 'accoa_c', 1.3303536033469967],
 ['3odcoa_c', 'actp_c', 56.57509098306293],
 ['3odcoa_c', 'fadh2_c', 56.57509098306293]]

In [17]:
df = pd.DataFrame(data=reduced_met_orders)
reduced_met_orders = 

## Prepare JSON file for Cytoscape.js

In [14]:
# Build data dictionary
data = {}
data['nodes'] = []
data['edges'] = []

for met in total_mets:
    data['nodes'].append(
    {
        'data': {
            'id': met,
            'label': met,
            'name': iJO1366.metabolites.get_by_id(met).name
        }
    })

for met_pairs in reduced_met_orders:
    data['edges'].append(
    {
        'data': {
            'id': met_pairs[0] + '_' + met_pairs[1],
            'source': met_pairs[0],
            'target': met_pairs[1],
            'label': str(np.round(met_pairs[2], 2))
        }
    })

# write to json
with open('data/conditions/graphData.json', 'w') as outfile:  
    json.dump(data, outfile)